In [1]:
"""
2025-02-10 MY
Query PAITULI STAC to find out NDVI mosaics.

NDVI Time-Series from Sentinel-2 Imagery Using STAC

"""

import stackstac
from dask.distributed import Client, Lock
import pystac_client
import pyproj
import geopandas as gpd
import json
import requests
import os
import rioxarray
import re
import pandas as pd

In [3]:
def change_to_https(request: requests.Request) -> requests.Request: 
    request.url = request.url.replace("http:", "https:")
    return request


no_of_workers = len(os.sched_getaffinity(0))
client = Client(n_workers=no_of_workers)

In [11]:
URL = "https://paituli.csc.fi/geoserver/ogc/stac/v1"
catalog = pystac_client.Client.open(URL, request_modifier=change_to_https)
#catalog

In [5]:
print(f"Title: {catalog.title or 'N/A'}")
print(f"Description: {catalog.description or 'N/A'}")

Title: Paituli STAC
Description: Paituli STAC with Finnish data. More info: https://paituli.csc.fi/stac.html


In [13]:
for collection in catalog.get_collections():
    #print(collection.id)
    if re.findall(r"(sentinel)[\._]", collection.id):
        print(collection.id)

sentinel_2_nir_b08_at_geocubes
sentinel_2_satellite_image_mosaic_at_geocubes
sentinel_2_swir_b11_at_geocubes
sentinel_2_11_days_mosaics_at_fmi
sentinel_1_daily_mosaics_at_fmi
sentinel_1_global_backscatter_at_geocubes
sentinel_2_annual_mosaics_at_fmi
sentinel_2_monthly_index_mosaics_at_fmi
sentinel_1_tiles_at_fmi
sentinel_1_11_days_mosaics_at_fmi


In [16]:
collection = catalog.get_collection('sentinel2-l2a')
#collection

In [17]:
%%time
AOI = [20.0, 59.3, 25.4, 61.3]

search_bbox = catalog.search(
    bbox=AOI,
    collections=["sentinel2-l2a"],
    datetime="2021-07-01/2021-08-15"
)
print('Found items: ' "{}".format(search_bbox.matched()))

Found items: 458
CPU times: user 21.8 ms, sys: 2 ms, total: 23.8 ms
Wall time: 356 ms
